### **Data Reading**

In [0]:
df= spark.read.format("parquet")\
  .load("abfss://bronze@databrickssk.dfs.core.windows.net/orders")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df=df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df=df.drop("rescued_data")

In [0]:

df=df.withColumn("year",year("order_date"))
df.display()

In [0]:
df1=df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1=df1.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

df1.display()

### **Classes - OOP**

In [0]:
class windows:
    def __init__(self,df):
        self.df=df
    def dense_rank(self):
        df_dense_rank=self.df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    def rank(self):
        df_rank=self.df.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    def row_number(self):
        df_row_number=self.df.withColumn("row_number",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number
    

In [0]:
df_new=df

df_new=windows(df_new).dense_rank()
df_new=windows(df_new).rank()
df_new=windows(df_new).row_number()
df_new.display()

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databrickssk.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databrickssk.dfs.core.windows.net/orders'